In [5]:
# Install required packages
!pip install --upgrade pythreejs ipywidgets numpy matplotlib Pillow plotly
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager pythreejs

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [6]:
%%html
<div id="container"></div>
<div id="info">
  <a href="https://threejs.org" target="_blank" rel="noopener">three.js</a> webgl - golden ratio spiral
</div>

<script type="module">

  import * as THREE from 'https://unpkg.com/three?module';
  import {
    BufferGeometry,
    Color,
    Float32BufferAttribute,
    PerspectiveCamera,
    Scene,
    TextureLoader,
    PointsMaterial,
    Points,
    WebGLRenderer,
    AdditiveBlending
  } from "https://unpkg.com/three?module";

  let container;
  let camera, scene, renderer;
  let points, geometry;
  let startTime;
  const galaxyImage = "https://cdn.pixabay.com/photo/2023/01/10/10/47/space-7709489_1280.jpg";
  const starTextureURL = "https://threejs.org/examples/textures/sprites/spark1.png";

  init();
  animate();

  function init() {
    container = document.getElementById("container");

    // Set up the camera
    let aspectRatio = window.innerWidth / window.innerHeight;
    camera = new PerspectiveCamera(45, aspectRatio, 1, 5000 );
    camera.position.z = 1000;

    scene = new Scene();

    // Load galaxy background
    const loader = new TextureLoader();
    loader.load( galaxyImage, function ( texture ) {
      scene.background = texture;
    } );

    // Load star texture and create the spiral once texture is loaded
    const starTextureLoader = new TextureLoader();
    starTextureLoader.load( starTextureURL, function ( starTexture ) {

      // Create golden spiral points
      const segments = 10000;

      const positions = [];
      const colors = [];

      const phi = (1 + Math.sqrt( 5 )) / 2; // Golden ratio
      const b = Math.log(phi) / (Math.PI / 2);
      const a = 50;
      const c = 20;

      const theta_max = 20 * Math.PI; // Adjust to get more turns

      for ( let i = 0; i < segments; i ++ ) {

        const theta = i / segments * theta_max;

        const r = a * Math.exp( b * theta );

        const x = r * Math.cos( theta );
        const y = r * Math.sin( theta );
        const z = -c * theta; // Negative to position spiral towards the camera

        positions.push( x, y, z );

        // Set color based on theta or any other parameter
        const color = new Color();
        color.setHSL( ( theta / theta_max ), 1.0, 0.5 );
        colors.push( color.r, color.g, color.b );
      }

      geometry = new BufferGeometry();
      geometry.setAttribute( 'position', new Float32BufferAttribute( positions, 3 ) );
      geometry.setAttribute( 'color', new Float32BufferAttribute( colors, 3 ) );

      // Store initial positions for animation
      geometry.setAttribute( 'initialPosition', new THREE.Float32BufferAttribute( positions.slice(), 3 ) );

      // Create Points material with star texture
      const material = new THREE.PointsMaterial( {
        size: 10,
        map: starTexture,
        vertexColors: true,
        blending: THREE.AdditiveBlending,
        transparent: true,
        depthTest: false,
        opacity: 0.9
      } );

      // Create Points object
      points = new THREE.Points( geometry, material );
      scene.add( points );

      // Start the animation timer after everything is set up
      startTime = Date.now();

    } );

    // Renderer
    renderer = new THREE.WebGLRenderer( { antialias: true } );
    renderer.setPixelRatio( window.devicePixelRatio );
    renderer.setSize( window.innerWidth, window.innerHeight );

    container.appendChild( renderer.domElement );

    window.addEventListener( 'resize', onWindowResize, false );

  }

  function onWindowResize() {

    camera.aspect = window.innerWidth / window.innerHeight;
    camera.updateProjectionMatrix();

    renderer.setSize( window.innerWidth, window.innerHeight );

  }

  function animate() {

    requestAnimationFrame( animate );

    render();

  }

  function render() {

    if ( points ) {

      const elapsedTime = ( Date.now() - startTime ) * 0.001; // Time in seconds since animation started

      // Update positions to make the spiral move away from the camera
      const positions = geometry.attributes.position.array;
      const initialPositions = geometry.attributes.initialPosition.array;

      const speed = 50; // Adjust speed to control movement

      for ( let i = 0; i < positions.length; i += 3 ) {

        const x0 = initialPositions[ i ];
        const y0 = initialPositions[ i + 1 ];
        const z0 = initialPositions[ i + 2 ];

        // Move the point along the z-axis away from the camera
        positions[ i ] = x0;
        positions[ i + 1 ] = y0;
        positions[ i + 2 ] = z0 - elapsedTime * speed;

      }

      geometry.attributes.position.needsUpdate = true;

      // Optional rotation for added effect
      points.rotation.z += 0.001;

    }

    renderer.render( scene, camera );

  }

</script>